# Машинное обучение ранжированию с помощью библиотеки XGBoost

В этом примере мы:

- познакомимся с библиотекой **XGBoost**
- научимся решать задачу машинного обучения ранжирования используя алгоритмы, реализованные в **XGBoost**

Будем использовать стандартный датасет [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)

Подробности про формат датасета можно найти в соседнем тюториале *catboost_ltr.ipynb* по машинному обучению ранжированию с помощью библиотеки **CatBoost**, предполагаем что мы его уже прошли.

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [ ]:
import copy
from timeit import default_timer as timer

import xgboost as xgb
# We'll use catboost to download dataset and to compute metrics
from catboost import datasets, utils

## Датасет MSLR (Microsoft Learning to Rank)

Загрузим встроенный в катбуст сабсет датасета MSLR:

In [ ]:
df_train, df_test = datasets.msrank_10k()

Для удобства присвоим колонкам говорящие имена:

In [ ]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns

# Assign column names
columns = generate_column_names(num_features=136)
df_train.columns = columns
df_test.columns = columns

Теперь наши данные выглядят красивее:

In [ ]:
print(df_train.head(5))

Всего у нас:

- 87 запросов и 10000 документов в трейне
- 88 запросов и 10000 документов в тесте
- 130 колонок: таргет (оценка асессора), id запроса (qid) и вектор из 128 фичей
- таргет принимает значения в интервале \[0,4\]

Теперь нам надо представить датасет в формате, который можно подавать на вход модели xgboost.  
По полной аналогии с тем как мы это делали для catboost, придется разделить его на 3 части:

- **y** -- вектор таргетов
- **X** -- тензор из фичей
- **q** -- вектор из id запросов, которые позволяют сгруппировать все документы, которые относятся к одному и тому же запросу

Заметим, что xgboost, в отличие от catboost, требует строгой отсортированности по id запроса. Однако в данном случае нам повезло и датасет уже и так отсортирован по qid.

Убедимся в этом:

In [ ]:
print(df_train['qid'].is_monotonic_increasing)
print(df_test['qid'].is_monotonic_increasing)

Сконвертируем датасет в нужный формат:

In [ ]:
def to_xgboost_dataset(df):
    y = df['label'].to_numpy()                       # Label: [0-4]
    q = df['qid'].to_numpy().astype('uint32')        # Query Id
    X = df.drop(columns=['label', 'qid']).to_numpy() # 136 features
    return (X, y, q)

X_train, y_train, q_train = to_xgboost_dataset(df_train)
X_test, y_test, q_test = to_xgboost_dataset(df_test)

## Обучаем pointwise модель

Теперь можно приступить непосредственно к обучению модели. Мы начнем с простой pointwise модели которая в качестве лосса использует обычное RMSE.

Количество деревьев будем выбирать автоматически с использованием т.н. early stopping -- процесс обучения будем остановлен после того, как на валидационном множестве перестанет расти наша целевая метрика (в нашем случае это NDCG@10).

Создадим объект модели:

In [ ]:
model = xgb.XGBRegressor(
    n_estimators=1000,            # maximum possible number of trees
    objective='reg:squarederror', # RMSE
    eval_metric=['ndcg@10'],      # metric used for early stopping
    early_stopping_rounds=10,     # stop if eval_metric does not improve for N rounds
    random_state=22,
)

И зафитим ее на нашем обучающем множестве.  
Т.к. мы используем early stopping то передадим в функцию fit() в качестве валидационного множества (параметр eval_set) наш тест-сет (строго говоря так делать нельзя, но мы в данном примере не стремимся к строгости, подробнее этот момент расписан в тюториале про машинное обучение ранжированию с помощью каибуста).

In [ ]:
# Fit
start = timer()
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=True)
elapsed = timer() - start
print(f"model fit: elapsed = {elapsed:.3f}")

Посмотрим, сколько деревьев содержит модель. Это немного сложнее чем было с катбустом:

In [ ]:
num_trees = len(model.get_booster().get_dump())
best_iteration = model.best_iteration
best_score = model.best_score
print(f"model params: num_trees = {num_trees} best_iteration = {best_iteration} best_score = {best_score:.3f}")

Видно, что всего модель содержит 13 деревьев, но лучший скор на валидации достигается уже после применения 2х из них.  
Поэтому на этапе инференса будет использоваться всего 2 дерева.

При желании, мы можем сохранить эту модель в формате json:

In [ ]:
model_file = "/tmp/model.xgb.json"

# Save model
model.save_model(model_file)

# Load model
# model = xgb.XGBRegressor()
# model.load_model(model_file)

Получим предикты модели на тестовом множестве:

In [ ]:
y_hat_test = model.predict(X_test)
print(f"predicted: y_hat_test.shape = {y_hat_test.shape}")

Теперь, имея предикты, можно посчитать метрики качества, тут для удобства и единообразия будем использовать catboost:

In [ ]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp', 'DCG:top=10;type=Exp', 'MAP:top=10']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

Видим, что значение NDCG на тесте получилось значительно хуже, чем у катбуста (но, при желании, можно сделать сильно лучше если потюнить параметры модели).

## Обучаем pairwise модели

Теперь проделаем все то же самое, но с использованием алгоритма RankNet.  

Для этого вместо *XGBRegressor* будем использовать класс *XGBRanker*, которому в качестве objective передадим 'rank:pairwise'.

Также, теперь мы должны передавать в метод fit() наши вектора q_train и q_test, чтобы модель могла сгруппировать документы по запросу:

In [ ]:
# Create model
model = xgb.XGBRanker(
    n_estimators=1000,          # maximum possible number of trees
    objective='rank:pairwise',  # RankNet objective
    eval_metric=["ndcg@10"],    # metric used for early stopping
    early_stopping_rounds=10,   # stop if eval_metric does not improve for N rounds
    random_state=22,
)

# Fit
start = timer()
model.fit(X_train, y_train, qid=q_train, eval_set=[(X_test, y_test)], eval_qid=[q_test], verbose=True)
elapsed = timer() - start
print(f"model fit: elapsed = {elapsed:.3f} num_trees = {model.best_iteration}")

# Predict
y_hat_test = model.predict(X_test)

# Compute metrics on test
print("\nevaluated:")
compute_metrics(y_test, y_hat_test, q_test)

Сравним результаты:

- RMSE модель выбила NDCG@10 = 0.353
- а RankNet выбивает уже NDCG@10 = 0.387

Мы опять видим преимущество listwise-подхода над "наивным" pointwise.

И, наконец, попробуем полноценный LambdaRank, для этого:

- в качестве objective будем использовать значение 'rank:ndcg'
- будем использовать дополнительные параметры lambdarank_pair_method и lambdarank_num_pair_per_sample (про них можно почитать в [документации](https://xgboost.readthedocs.io/en/latest/tutorials/learning_to_rank.html))

In [ ]:
# Create model
model = xgb.XGBRanker(
    n_estimators=1000,          # maximum possible number of trees
    objective='rank:ndcg',      # LambdaRank objective
    lambdarank_pair_method='topk',
    lambdarank_num_pair_per_sample=20,
    eval_metric=["ndcg@10"],    # metric used for early stopping
    early_stopping_rounds=10,   # stop if eval_metric does not improve for N rounds
    random_state=22,
)

# Fit
start = timer()
model.fit(X_train, y_train, qid=q_train, eval_set=[(X_test, y_test)], eval_qid=[q_test], verbose=True)
elapsed = timer() - start
print(f"model fit: elapsed = {elapsed:.3f} num_trees = {model.best_iteration}")

# Predict
y_hat_test = model.predict(X_test)

# Compute metrics on test
print("\nevaluated:")
compute_metrics(y_test, y_hat_test, q_test)

Еще раз сравним результаты:

- RMSE модель выбила NDCG@10 = 0.353
- RankNet выбивает уже NDCG@10 = 0.387
- LambdaRank лучше всех и выбивает 0.416

Однако, обратим внимание, что, несмотря на то, что pairwise подходы опять победили, абсолютные значения NDCG@10 даже у LambdaRank хуже того, что выбивала простая регрессия в catboost (0.419), не говоря уже от YetiRank!

Видно, что "из коробки" catboost показывает значительно лучшие результаты, чем xgboost.  
Справедливости ради заметим, что мы в наших примерах совсем не пытались тюнить гиперпараметры -- если это проделать, то из xgboost можно "выжать" скоры значительно лучше.  
Тем не менее, по нашему опыту, на данный момент catboost действительно показывает на задаче ранжирования результаты лучше "конкурентов".